In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Fetched 261 kB in 3s (96.2 kB/s)
Reading package li

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 00:07:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.67MB/s    in 0.5s    

2022-11-30 00:07:30 (1.67 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
appliances_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
appliances_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [4]:
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date
Review_ID_a_df = appliances_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
Review_ID_a_df.withColumn("review_date",to_date(col('review_date'), 'yyyy-mm-dd'))
Review_ID_a_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R203HPW78Z7N4K|   16199106|B0067WNSZY|     633038551| 2015-08-31|
|R2EAIGVLEALSP3|   16374060|B002QSXK60|     811766671| 2015-08-31|
|R1K1CD73HHLILA|   15322085|B00EC452R6|     345562728| 2015-08-31|
|R2KZBMOFRMYOPO|   32004835|B00MVVIF2G|     563052763| 2015-08-31|
| R6BIZOZY6UD01|   25414497|B00IY7BNUW|     874236579| 2015-08-31|
|R1MCXZFNF8E7Y0|   36311751|B0033X29CI|     294467812| 2015-08-31|
|R3EMB3E3ODR6BW|   30920961|B005R597HA|     183784715| 2015-08-31|
| RJTONVTTOPJ5S|   52491265|B00MO6V8Y0|     960251524| 2015-08-31|
|R21U5QZ2CQECUM|   48166169|B00HT39QDI|     992475314| 2015-08-31|
| RL2BBC51H89DH|   50394924|B00LESFZ52|       1641606| 2015-08-31|
|R3RNEPHF3WIRSZ|    3915552|B0149IJVPI|     838108342| 2015-08-31|
|R38DNT9KML2PF3|   17068589|B002HT0958|     387104338| 2015-08

In [11]:
Review_ID_a_df.count()

96901

In [5]:
Product_ID_a_df = appliances_df.select(["product_id","product_title"])
Product_ID_a_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0067WNSZY|FGGF3032MW Galler...|
|B002QSXK60|Best Hand Clothes...|
|B00EC452R6|Supco SET184 Ther...|
|B00MVVIF2G|Midea WHS-160RB1 ...|
|B00IY7BNUW|Avalon Bay Portab...|
|B0033X29CI|Danby  Freestandi...|
|B005R597HA|Avanti 110-Volt A...|
|B00MO6V8Y0|      Danby products|
|B00HT39QDI|3 Pack Tier1 MSWF...|
|B00LESFZ52|True TSSU-60-16 6...|
|B0149IJVPI|Magic: the Gather...|
|B002HT0958|Mini Portable Cou...|
|B006WOBNX6|1 X Dishwasher Ra...|
|B00NLPMOU0|Watson 1.5 ft AC ...|
|B00KJ07SEM|GE MWF SmartWater...|
|B001AT2ALM|Koldfront Ultra C...|
|B00JG8B42K|OnePurify RFC0800...|
|B00FRJ5EWS|2016 Life Water I...|
|B0052G14E8|Danby 0.7 cu.ft. ...|
|B00LOVVE9A|SPT UF-304SS Ener...|
+----------+--------------------+
only showing top 20 rows



In [12]:
Product_ID_a_df.count()

96901

In [6]:
# Create a Customers Table with columns Customer_ID, and Customer_Count
Customers_a_df = appliances_df.groupby("customer_id").agg({"customer_id":"count"})
Customers_a_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   13326061|                 1|
|   52512151|                 1|
|    8968933|                 1|
|   39416583|                 1|
|   21737233|                 1|
|   44119972|                 1|
|   13947800|                 1|
|   21482558|                 1|
|    2802853|                 1|
|   18518845|                 1|
|     122484|                 1|
|   14035069|                 1|
|   20004191|                 1|
|   16699322|                 1|
|   24768141|                 1|
|   24424556|                 1|
|   44293588|                 1|
|   13188682|                 1|
|   23298840|                 1|
|   50731398|                 1|
+-----------+------------------+
only showing top 20 rows



In [13]:
Customers_a_df.count()

91311

In [8]:
# Vine Table with columns Review_ID, Star_Rating, Helpful_Votes, Total_Votes, and Vine
Vine_a_df = appliances_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
Vine_a_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R203HPW78Z7N4K|          5|            0|          0|   N|
|R2EAIGVLEALSP3|          5|            1|          1|   N|
|R1K1CD73HHLILA|          5|            0|          0|   N|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|
| R6BIZOZY6UD01|          5|            0|          0|   N|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|
| RJTONVTTOPJ5S|          5|            0|          0|   N|
|R21U5QZ2CQECUM|          4|            0|          0|   N|
| RL2BBC51H89DH|          4|            0|          0|   N|
|R3RNEPHF3WIRSZ|          2|            0|          0|   N|
|R38DNT9KML2PF3|          5|            0|          0|   N|
|R2ECMBJM8KNNC8|          4|            0|          0|   N|
|R2F3F92PRN9T7S|          5|            

In [14]:
Vine_a_df.count()

96901

In [9]:
# Load into Postgres
#Configure Postgres and Colab together
mode = "append"
jdbc_url="jdbc:postgresql://appliance-db.cfiruawf1lll.us-east-1.rds.amazonaws.com:5432/appliance_reviews_db"
config = {"user":"postgres", "password": "", "driver":"org.postgresql.Driver"}

In [10]:
#redo four times one time for each table
Review_ID_a_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
Product_ID_a_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
Customers_a_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
Vine_a_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

There is a similar number of customers vs reviews suggesting if someone buys an appliance there is a good chance that they will write a review on it.